In [ ]:
import xumm
import json
import asyncio
import nest_asyncio


async def send_payment_request(amount, currency, source, destination, payment_reference):
    
    # Create the XUMM payment request payload
    xumm_payload = {
        "txjson": {
            "TransactionType": "Payment",
            "Account": source,
            "Destination": destination,
            "Amount" : str(amount),
        },
        "Fee":"12",
        "options": {
            "expire": 1
        },
        "user_token":"f53f2b43-a000-47fc-bcd8-d4f5acf9d234"
    }
    
    def callback_func(event):
        print('New payload event: {}'.format(event['data']))

        if 'signed' in event['data']:
            print('Signed')
            return event['data']

    # Create the payment request with the XUMM SDK
    sdk = xumm.XummSdk('dc4acbcf-2841-4bea-aafc-ee2fbfc5ec64', 'c50fc9f2-9a9b-4981-a021-d64b41389203')
    pong = sdk.ping()
    print(pong)
    subscription = await sdk.payload.create_and_subscribe(
      xumm_payload,
      callback_func,
    )

    # Print the payment request URL
    print(json.dumps(subscription.created.to_dict(), indent=4, sort_keys=True))
    print('New payload created, URL: {}'.format(subscription.created.next.always))
    print('  > Pushed: {}'.format('yes' if subscription.created.pushed else 'no'))

    """
    Wait until the subscription resolves (by returning something)
    in the callback function.
    """
    resolve_data = await subscription.resolved()
    print(resolve_data)
    try:
        if resolve_data['signed'] == False:
            print('The sign request was rejected.')

        else:
            print('The sign request was approved.')
            """
            Fetch the full payload end result, and get the issued
            user token, we can use to send our next payload per Push notification
            """
            result = sdk.payload.get(resolve_data['payload_uuidv4'])
            print('User token: {}'.format(result.application.issued_user_token))
        
    except:
        print('Error')

In [ ]:
# Apply the nest_asyncio patch
nest_asyncio.apply()

# Run the send_payment_request coroutine
loop = asyncio.get_event_loop()
loop.run_until_complete(send_payment_request(100000, 'XRP', 'rB4iz44nvW2yGDBYTkspVfyR2NMsR3NtfF', 'rpaxHGQVgQSXF1HaKGRpLKm6X7eh26v6eV', 'test payment request'))

In [ ]:
send_payment_request(10000, 'XRP', 'rB4iz44nvW2yGDBYTkspVfyR2NMsR3NtfF', 'rMakHZJfto1GAkVPYxkFLZwzrNmkcgvW62', 'test payment request')

In [ ]:
('dc4acbcf-2841-4bea-aafc-ee2fbfc5ec64', 'c50fc9f2-9a9b-4981-a021-d64b41389203')

In [ ]:
{
    "next": {
        "always": "https://xumm.app/sign/3bbba11b-7481-4994-b862-557dc3198460"
    },
    "pushed": false,
    "refs": {
        "qr_matrix": "https://xumm.app/sign/3bbba11b-7481-4994-b862-557dc3198460_q.json",
        "qr_png": "https://xumm.app/sign/3bbba11b-7481-4994-b862-557dc3198460_q.png",
        "qr_uri_quality_opts": [
            "m",
            "q",
            "h"
        ],
        "websocket_status": "wss://xumm.app/sign/3bbba11b-7481-4994-b862-557dc3198460"
    },
    "uuid": "3bbba11b-7481-4994-b862-557dc3198460"
}

In [1]:
import xumm
import json
import asyncio
import nest_asyncio

async def send_payment_request(amount, currency, source, destination, payment_reference):
    
    # Create the XUMM payment request payload
    xumm_payload = {
        "txjson": {
            "TransactionType": "Payment",
            "Account": source,
            "Destination": destination,
            "Amount" : str(amount),
        },
        "Fee":"12",
        "options": {
            "expire": 3
        },
        "user_token":"f53f2b43-a000-47fc-bcd8-d4f5acf9d234"
    }
    
    # Create the payment request with the XUMM SDK
    sdk = xumm.XummSdk('dc4acbcf-2841-4bea-aafc-ee2fbfc5ec64', 'c50fc9f2-9a9b-4981-a021-d64b41389203')
    try:
        subscription = sdk.payload.create(xumm_payload)
        print(json.dumps(subscription.to_dict(), indent=4, sort_keys=True))
        print('New payload created, URL: {}'.format(subscription.next.always))
    except Exception as e:
        print(f"Error creating subscription: {e}")
        # Handle the error as appropriate
        
# async def main():
#     amount = 10000
#     currency = "XRP"
#     source = "rB4iz44nvW2yGDBYTkspVfyR2NMsR3NtfF"
#     destination = "rMakHZJfto1GAkVPYxkFLZwzrNmkcgvW62"
#     payment_reference = "test payment request"

# #     task = asyncio.create_task(send_payment_request(amount, currency, source, destination, payment_reference))
# #     await task
#     await send_payment_request(amount, currency, source, destination, payment_reference)

# if __name__ == "__main__":
#     await main()

In [2]:
# Apply the nest_asyncio patch
nest_asyncio.apply()

# Run the send_payment_request coroutine
loop = asyncio.get_event_loop()
loop.run_until_complete(send_payment_request(100000, 'XRP', 'rB4iz44nvW2yGDBYTkspVfyR2NMsR3NtfF', 'rpaxHGQVgQSXF1HaKGRpLKm6X7eh26v6eV', 'test payment request'))

{
    "next": {
        "always": "https://xumm.app/sign/42f8d56d-759b-4856-bbac-685b3448ac5c"
    },
    "pushed": true,
    "refs": {
        "qr_matrix": "https://xumm.app/sign/42f8d56d-759b-4856-bbac-685b3448ac5c_q.json",
        "qr_png": "https://xumm.app/sign/42f8d56d-759b-4856-bbac-685b3448ac5c_q.png",
        "qr_uri_quality_opts": [
            "m",
            "q",
            "h"
        ],
        "websocket_status": "wss://xumm.app/sign/42f8d56d-759b-4856-bbac-685b3448ac5c"
    },
    "uuid": "42f8d56d-759b-4856-bbac-685b3448ac5c"
}
New payload created, URL: https://xumm.app/sign/42f8d56d-759b-4856-bbac-685b3448ac5c


In [ ]:
'dc4acbcf-2841-4bea-aafc-ee2fbfc5ec64', 'c50fc9f2-9a9b-4981-a021-d64b41389203'

In [ ]:
async def getPayload():
    sdk = xumm.XummSdk('dc4acbcf-2841-4bea-aafc-ee2fbfc5ec64', 'c50fc9f2-9a9b-4981-a021-d64b41389203')
    subscription = sdk.payload.get('a4ead1c6-59c2-4d76-b5f1-6fd9be539e7f')
    print(json.dumps(subscription.to_dict()))
    print('New payload created, URL: {}'.format(subscription.created.next.always))

In [ ]:
# Apply the nest_asyncio patch
nest_asyncio.apply()

# Run the send_payment_request coroutine
loop = asyncio.get_event_loop()
loop.run_until_complete(getPayload())